## <CENTER><U> 1. IMPORTING BASIC NECESSARY PACKAGES</U></CENTER>

In [ ]:
pip install geopy

In [ ]:
import pandas as pd
import time
from geopy.distance import geodesic

## <CENTER><U> 2. READING INPUT DATA </CENTER></U>

In [ ]:
data = pd.read_csv("trajectory_data.csv")

In [ ]:
data.isnull().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates(keep='last').reset_index(drop=True)
data.duplicated().sum()

In [ ]:
display(data.head(10))

In [ ]:
display(data.tail(10))

In [ ]:
data.dtypes

In [ ]:
data.shape

In [ ]:
start_time = time.time()

pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None)
 
travel_count = 1

data['Travel Count'] = 1

data['Time'] = pd.to_datetime(data['Time'])

count = []

### Remove records which suggest Different modes during same time period

In [ ]:
# For each time point of track data, if it corresponds to multiple modes of transportation,
#all of them will be deleted

for i in range(len(data)-1):

    if (data['Time'][i + 1] - data['Time'][i]).seconds == 0:

        count.append(i)

        count.append(i+1)

data = data.drop(count).reset_index(drop=True)

count.clear()

### Remove abnormal latitude and longitude

In [ ]:
# Delete data with abnormal latitude and longitude

for i in range(len(data)):

    if data['Lat'][i] > 90 or data['Lat'][i] < 0 or data['Lon'][i] > 180 or data['Lon'][i] < -180:

        count.append(i)

data = data.drop(count).reset_index(drop=True)

### Splitting travel segments

In [ ]:
for i in range(len(data)):
    data['Travel Count'].at[i] = travel_count   
    if (i != len(data)-1):        
        if ((data['Travel Start Time'][i] != data['Travel Start Time'][i+1]) or \
            ((data['Time'][i + 1] - data['Time'][i]).seconds > 1800)): 
                
                travel_count = travel_count + 1
 
result = pd.value_counts(data['Travel Count'])

In [ ]:
end_time = time.time()
 
print(f'Time taken for processing the data:\t {end_time - start_time}')

## <center> <u> 4. ADDITIONAL FEATURE CALCULATION

In [ ]:
time_gap = [] 

distance_gap = []

speed = []

acceleration = []

total_time = []

total_time_count = 0

total_distance = []

total_distance_count = 0

In [ ]:
i = 0
count = []
for i in range(0,len(data)-1):
    if (data['Travel Count'][i] == data['Travel Count'][i+1])  and \
    ((data['Time'][i+1] - data['Time'][i]).seconds < 50):
        count.append(i+1)
        data['Time'].at[i+1] = data['Time'].at[i]
data = data.drop(count).reset_index(drop=True)

In [ ]:
for i in range(len(data)-1):

    if data['Travel Count'][i] == data['Travel Count'][i+1]:
        time_gap.append((data['Time'][i + 1] - data['Time'][i]).seconds)
        distance_gap.append(geodesic((data['Lat'][i], data['Lon'][i]), (data['Lat'][i + 1], data['Lon'][i + 1])).m)
    else:
        time_gap.append('N.A')
        distance_gap.append('N.A')
        
time_gap.append('NA')
distance_gap.append('N.A')

In [ ]:
for i in range(len(data)-1):
    if time_gap[i] != 'N.A':
        speed.append(round(distance_gap[i]/time_gap[i], 2))
    else:
        speed.append('N.A')      
speed.append('N.A')

In [ ]:
for i in range(len(data)-1):
    if speed[i] != 'N.A' and speed[i+1] != 'N.A':
        acceleration.append(round(((speed[i+1]-speed[i])/time_gap[i]), 2))
    else:
        acceleration.append('N.A')
acceleration.append('N.A')

In [ ]:
for i in range(len(data)-1):
    if time_gap[i] != 'N.A':
        total_time_count = total_time_count + time_gap[i]
        total_time.append('N.A')
        total_distance_count = total_distance_count + distance_gap[i]
        total_distance.append('N.A')
    else:
        total_time.append(total_time_count)
        total_distance.append(total_distance_count)
        total_time_count = 0
        total_distance_count = 0      
total_time.append('N.A')
total_distance.append('N.A')

In [ ]:
data['Time Gap(s)'] = time_gap
data['Distance(m)'] = distance_gap
data['Speed(m/s)'] = speed
data['Acceleration(m/s^2)'] = acceleration
data['Total Time(s)'] = total_time
data['Total Distance(m)'] = total_distance

In [ ]:
data.to_csv("Data processing/feature_caclculated.csv")